 ### Existing code from the online assignment

In [162]:
import os
import random
import re
import sys

DAMPING = 0.85
SAMPLES = 10000
TOLERANCE = 0.001

In [2]:
def crawl(directory):
    """
    Parse a directory of HTML pages and check for links to other pages.
    Return a dictionary where each key is a page, and values are
    a list of all other pages in the corpus that are linked to by the page.
    """
    pages = dict()

    # Extract all links from HTML files
    for filename in os.listdir(directory):
        if not filename.endswith(".html"):
            continue
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            links = re.findall(r"<a\s+(?:[^>]*?)href=\"([^\"]*)\"", contents)
            pages[filename] = set(links) - {filename}

    # Only include links to other pages in the corpus
    for filename in pages:
        pages[filename] = set(
            link for link in pages[filename]
            if link in pages
        )

    return pages

In [205]:
def transition_model(corpus, page, damping_factor):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """
    
    m = len(corpus[page]) # links out of the page
    n = len(corpus) # number of pages 
    p = 1 - damping_factor
    
    if m == 0:
        temp = {key:1/n for key in corpus}
        return temp
    
    model = {}
    
    for page in corpus.keys():
        model[page] = p * 1/n
    
    for link in corpus[page]:
        model[link] += damping_factor*1/m
    
    summation = 0
    for _,val in model.items():
        summation += val
    #print(summation)
    return model

In [206]:
def sample_pagerank(corpus, damping_factor, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """

    num_pages = len(corpus)
    page_names = list(corpus.keys())
    counts = {}
    
    for page in corpus:
        counts[page] = 0
        
    first_page = page_names[random.randint(0, num_pages-1)]
    counts[first_page] += 1
        
    sample = transition_model(corpus, first_page, damping_factor)
    #print(sample)
    for _ in range(n-1):
        keys = [key for key, _ in sample.items()]
        vals = [vals for  _, vals in sample.items()]
        
        rand_page = random.choices(population=keys, weights=vals, k=1)[0]
        counts[rand_page] += 1
        
        sample = transition_model(corpus, rand_page, damping_factor)
    
    for page in counts:
        counts[page] *= 1/n
    return counts

In [186]:
def iterate_pagerank(corpus, damping_factor):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    # Initialize page ranks at 1/N
    num_pages = len(corpus)
    page_names = corpus.keys()
    ranks = {key: (1 / num_pages) for key in page_names}

    # Make pages with no links link to every page
    for key in page_names:
        if (len(corpus[key]) == 0):
            corpus[key] = set(page_names)

    # Reorganize a new dict where each key: page has a value: set of pages linking to it
    has_links_in = {}
    for key in page_names:
        linking_pages = set([link for link in page_names if key in corpus[link]])
        has_links_in[key] = linking_pages
    
    while True:
        # Calculate the next iteration
        new_ranks = {}
        for key in page_names:
            sum = 0
            if (len(has_links_in[key]) > 0):
                for i in has_links_in[key]:
                    sum += ranks[i] / len(corpus[i])
            
            new_rank = ((1-damping_factor) / num_pages) + (damping_factor * sum)
            new_ranks[key] = new_rank

        # Find the difference between old and new pagerank
        diff = [abs(a - b) for (a, b) in zip(ranks.values(), new_ranks.values())]
        ranks = new_ranks
        # Loop until it converges to within TOLERANCE
        if (max(diff) < TOLERANCE):
            break

    return ranks

In [207]:
def main():

    corpus = crawl("testdirectory/corpus2")
    #print(corpus)
    #print(transition_model(corpus, '1.html', DAMPING)) 
    ranks = sample_pagerank(corpus, DAMPING, SAMPLES)
    print(f"PageRank Results from Sampling (n = {SAMPLES})")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")
    ranks = iterate_pagerank(corpus, DAMPING)
    print(f"PageRank Results from Iteration")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")


main()

AttributeError: 'set' object has no attribute 'keys'